In [11]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
# from dash import ctx
import numpy as np

['Tuần 27.02-05.03',
 'Tuần 06.03-12.03',
 'Tuần 13.03-19.03',
 'Tuần 20.03-26.03',
 'Tuần 27.03-01.04']

['Hub', 'Rm', 'Rm Name', 'Team', 'Phân Khúc', 'Đã Phân Bổ', 'Đã Gọi',
       'Đã Gặp', 'Thu Thập Hồ Sơ', 'Định Giá', 'Đã Trình', 'Đã Phê Duyệt',
       'Đã Active Hạn Mức'],

In [12]:
path = 'Hoạt động bán hàng All RM Tháng 3 theo tuần.xlsx'

In [13]:
def get_data(path):
  sheets = pd.ExcelFile(path)
  toan_quoc = []
  hubs = []
  fields = ['Hub', 'Đã Gọi','Đã Gặp', 'Thu Thập Hồ Sơ', 
          'Định Giá', 'Đã Trình', 'Đã Phê Duyệt','Đã Active Hạn Mức']
  for sheet_name in sheets.sheet_names:
    sheet = sheets.parse(sheet_name)
    sheet.columns = [x.strip().title() for x in sheet.columns]
    toan_quoc.append(sheet[sheet['Rm Name'].str.startswith('0.')].loc[:,fields[1:]])
    hubs_sheet = sheet[sheet['Rm Name'].str.startswith('1.')].sort_values(by=['Đã Active Hạn Mức'], ascending=False).loc[:,fields]
    hubs.append(hubs_sheet)
    # hubs.append(hubs_sheet.loc[(hubs_sheet[fields[1:]]!=0).any(axis=1)])
  hub_names = hubs[0].sort_values(by=['Hub']).Hub.values.tolist()
  return toan_quoc, hubs, sheets.sheet_names, hub_names

In [14]:
toan_quoc, hubs, weeks, hub_names= get_data(path)
#toan_quoc: data la mot danh sach cac df 1 row theo tuan
# hubs : data la danh sach data cua hub theo tuan
# week: danh sach tuan 
# danh sach hub theo alphabet

In [15]:
state = ['Đã Gọi','Đã Gặp', 'Thu Thập Hồ Sơ','Định Giá', 'Đã Trình', 'Đã Phê Duyệt','Đã Active Hạn Mức']
color = ['#94FFFF','#66FFFF','#39FFFF','#0BFFFF','#00DCDC','#00AEAE','#008080']

# Toàn Quốc


## Stacked Funnel

In [16]:
fig = go.Figure()
fig.update_layout(
        plot_bgcolor='rgb(255,255,255)')
for i in range(len(weeks)):
  fig.add_trace(go.Funnel(
      name = weeks[i],
      x = toan_quoc[i].values.flatten(),
      y = state,
      # width = 1,
      # marker = {'color':color}
      ))
fig.show()

In [17]:
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

username = 'thutranghoa24'
api_key = 'UebMvoGgyVao9nhYCtTr'

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

first = fig
py.plot(first, filename = 'old', auto_open = False)

'https://plotly.com/~thutranghoa24/30/'

## With graph objects



In [18]:
fig = go.Figure()
fig.update_layout(plot_bgcolor='rgb(255,255,255)')
fig.add_trace(go.Funnel(
    x = toan_quoc[0].values.flatten(),
    y = state,
    # width = 1,
    marker = {'color':color}))

## With plotly.express

In [19]:
import plotly.express as px
fig = px.funnel(x=toan_quoc[0].values.flatten(), 
                y=state,
                color_discrete_sequence = px.colors.convert_colors_to_same_type(color)
)
fig.show()

## Chart in Dash

In [20]:
#graph pbjects
import dash
fig = go.Figure()
fig.update_layout(plot_bgcolor='rgb(255,255,255)')
fig.add_trace(go.Funnel(
    x = toan_quoc[0].values.flatten(),
    y = state,
    # width = 1,
    marker = {'color':color})
)

# # plotly.express 
# fig = px.funnel(x=toan_quoc[0].values.flatten(), 
#                 y=state,
#                 color_discrete_sequence = px.colors.convert_colors_to_same_type(color)
# )
# fig.update_layout(plot_bgcolor='rgb(255,255,255)')

# build a jupyter dash app 
app = dash.Dash(__name__)

# add html components and figure to app
app.layout = html.Div(
    [
      html.H1('Phễu bán hàng trên toàn quốc 27/02-05/03', style={'color': 'teal'}),
      dcc.Graph(figure=fig)
    ]
  )

# run app inline
app.run_server(debug = False) #"inline" will output the dashboard in colab, "external" will output it in your browser via localhost

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit


## Chart with dropdown 

In [25]:
import dash
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1('Phễu bán hàng trên toàn quốc 27/02-01/04', 
            style={'text-align': 'center', 'color': 'teal'}),
    html.Div([
        html.Label(['Choose a graph:'],style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='dropdown',
            options = [{'label':weeks[i],'value':str(i)} for i in range(len(weeks))],
            value='0',
            style={"width": "60%"}),
        html.Div(dcc.Graph(id='graph')),        
        ]),

])

@app.callback(
    Output('graph', 'figure'),
    [Input(component_id='dropdown', component_property='value')]
)
def select_graph(value):
    fig = go.Figure()
    fig.update_layout(
        plot_bgcolor='rgb(255,255,255)',
         margin=go.layout.Margin(
        l=10, #left margin
        r=10, #right margin
        b=10, #bottom margin
        t=10  #top margin
        )
    )
    fig.add_trace(go.Funnel(
        x = toan_quoc[int(value)].values.flatten(),
        y = state,
        # width = 1,
        marker = {'color':color})
    )
    return fig
    
app.run_server(debug = False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [20/Apr/2023 23:36:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 23:36:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 23:36:08] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 23:36:08] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 23:36:08] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Apr/2023 23:36:08] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Apr/2023 23:36:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 23:36:08] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Apr/2023 23:36:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 23:36:15] "POST /_dash-update-component HTTP/1.1" 200 -


#Hubs

phễu theo từng hub và thời gian 

In [26]:
hub_indexed=[]
for i in range(len(weeks)):
  hub_indexed.append(hubs[i].set_index('Hub'))
hub_indexed

[                Đã Gọi  Đã Gặp  Thu Thập Hồ Sơ  Định Giá  Đã Trình  \
 Hub                                                                  
 SSE Miền Bắc       373       9               4         5         7   
 Tân Thuận           49      19               3         4         4   
 Tân Bình            46      20               2         3         3   
 SSE Miền Nam        31       8               8         8         8   
 Bắc Ninh           135      23               3         3         2   
 ...                ...     ...             ...       ...       ...   
 Nam Thăng Long      33      29               6         6         5   
 Nam Định             7       5               0         1         1   
 Ninh Bình           43       5               1         0         0   
 SSE An Thới          0       0               0         0         0   
 Hà Nội              59      44               1         2         2   
 
                 Đã Phê Duyệt  Đã Active Hạn Mức  
 Hub                    

In [23]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('Phễu bán hàng của các hub trên toàn quốc 27/02-01/04', 
            style={'text-align': 'center', 'color': 'teal'}),
    html.Div([
        html.Label(['Chọn Thời gian'],style={'font-weight': 'bold'}),
        dcc.RadioItems(
             id='radio',
             options=[{'label': weeks[i], 'value': str(i)} for i in range(len(weeks))],
             value='0',
             inline=True, 
             labelStyle = {'cursor': 'pointer', 'margin-left':'20px'}),

        html.Label(['Chọn Hub:'],style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='dropdown',
            options = [{'label':hub,'value':hub} for hub in hub_names],
            value=hub_names[0],
            style={"width": "50%"}),
        ]),
        html.Div(dcc.Graph(id='graph')) 
])

@app.callback(
    Output('graph', 'figure'),
    [Input(component_id='dropdown', component_property='value'),
    Input(component_id='radio', component_property='value')]
)
def select_graph(value_d,value_r):
    fig = go.Figure()
    fig.update_layout(
        plot_bgcolor='rgb(255,255,255)',
         margin=go.layout.Margin(
        l=10, #left margin
        r=10, #right margin
        b=10, #bottom margin
        t=10  #top margin
        )
    )
    fig.add_trace(go.Funnel(
        x = hub_indexed[int(value_r)].loc[value_d],
        y = state,
        # width = 1,
        marker = {'color':color})
    )
    return fig
    
app.run_server(debug = False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [20/Apr/2023 21:25:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 21:25:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 21:25:22] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 21:25:22] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 21:25:22] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Apr/2023 21:25:22] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 21:25:22] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Apr/2023 21:25:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 21:25:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 21:25:23] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 21:25:23] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 21:25:23] "GET /_dash-compone

# Tham khảo 

## Funel

In [ ]:


fig = go.Figure(go.Funnel(
    y = ["Website visit", "Downloads", "Potential customers", "Requested price", "invoice sent"],
    x = [39, 27.4, 20.6, 11, 2], ))

fig.show()

In [ ]:
from plotly import graph_objects as go

fig = go.Figure(go.Funnel(
    y = ["Website visit", "Downloads", "Potential customers", "Requested price", "Finalized"],
    x = [39, 27.4, 20.6, 11, 2],
    textposition = "inside",
    textinfo = "value+percent initial",
    opacity = 0.65, marker = {"color": ["deepskyblue", "lightsalmon", "tan", "teal", "silver"],
    "line": {"width": [4, 2, 2, 3, 1, 1], "color": ["wheat", "wheat", "blue", "wheat", "wheat"]}},
    connector = {"line": {"color": "royalblue", "dash": "dot", "width": 3}})
    )

fig.show()

In [ ]:
from plotly import graph_objects as go

fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'Montreal',
    y = ["Website visit", "Downloads", "Potential customers", "Requested price"],
    x = [120, 60, 30, 20],
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Toronto',
    orientation = "h",
    y = ["Website visit", "Downloads", "Potential customers", "Requested price", "invoice sent"],
    x = [100, 60, 40, 30, 20],
    textposition = "inside",
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name = 'Vancouver',
    orientation = "h",
    y = ["Website visit", "Downloads", "Potential customers", "Requested price", "invoice sent", "Finalized"],
    x = [90, 70, 50, 30, 10, 5],
    textposition = "outside",
    textinfo = "value+percent total"))

fig.show()

## Area Funel 

In [ ]:
import plotly.express as px
fig = px.funnel_area(names=["The 1st","The 2nd", "The 3rd", "The 4th", "The 5th"],
                    values=[5, 4, 3, 2, 1])
fig.show()


In [ ]:
from plotly import graph_objects as go

fig = go.Figure(go.Funnelarea(
    text = ["The 1st","The 2nd", "The 3rd", "The 4th", "The 5th"],
    values = [5, 4, 3, 2, 1]
    ))
fig.show()

In [ ]:
from plotly import graph_objects as go

fig = go.Figure(go.Funnelarea(
      values = [5, 4, 3, 2, 1], text = ["The 1st","The 2nd", "The 3rd", "The 4th", "The 5th"],
      marker = {"colors": ["deepskyblue", "lightsalmon", "tan", "teal", "silver"],
                "line": {"color": ["wheat", "wheat", "blue", "wheat", "wheat"], "width": [0, 1, 5, 0, 4]}},
      textfont = {"family": "Old Standard TT, serif", "size": 13, "color": "black"}, opacity = 0.65))
fig.show()

In [ ]:
from plotly import graph_objects as go

fig = go.Figure()

fig.add_trace(go.Funnelarea(
    scalegroup = "first", values = [500, 450, 340, 230, 220, 110], textinfo = "value",
    title = {"position": "top center", "text": "Sales for Sale Person A in U.S."},
    domain = {"x": [0, 0.5], "y": [0, 0.5]}))

fig.add_trace(go.Funnelarea(
    scalegroup = "first", values = [600, 500, 400, 300, 200, 100], textinfo = "value",
    title = {"position": "top center", "text": "Sales of Sale Person B in Canada"},
    domain = {"x": [0, 0.5], "y": [0.55, 1]}))

fig.add_trace(go.Funnelarea(
    scalegroup = "second", values = [510, 480, 440, 330, 220, 100], textinfo = "value",
    title = {"position": "top left", "text": "Sales of Sale Person A in Canada"},
    domain = {"x": [0.55, 1], "y": [0, 0.5]}))

fig.add_trace(go.Funnelarea(
            scalegroup = "second", values = [360, 250, 240, 130, 120, 60],
            textinfo = "value", title = {"position": "top left", "text": "Sales of Sale Person B in U.S."},
            domain = {"x": [0.55, 1], "y": [0.55, 1]}))

fig.update_layout(
            margin = {"l": 200, "r": 200}, shapes = [
            {"x0": 0, "x1": 0.5, "y0": 0, "y1": 0.5},
            {"x0": 0, "x1": 0.5, "y0": 0.55, "y1": 1},
            {"x0": 0.55, "x1": 1, "y0": 0, "y1": 0.5},
            {"x0": 0.55, "x1": 1, "y0": 0.55, "y1": 1}])

fig.show()

##Dash in GgColab

https://colab.research.google.com/drive/16Q8cndS4JuIAQnz17ZsKuv5CDAt2yU6S?usp=sharing#scrollTo=z42YtxOHgo3l